In [1]:
import torch
import gradio as gr

from textwrap import fill
from IPython.display import Markdown, display

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    )

from langchain import PromptTemplate
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

import warnings
warnings.filterwarnings('ignore')
import os
from tqdm.autonotebook import trange
from EmbeddingsPDF import EmbeddingsPDF
from dotenv import load_dotenv
load_dotenv()
print(os.environ)


environ({'COMMAND_MODE': 'unix2003', 'CONDA_DEFAULT_ENV': 'chatbot', 'CONDA_EXE': '/Users/admin/opt/miniconda3/bin/conda', 'CONDA_PREFIX': '/Users/admin/opt/miniconda3/envs/chatbot', 'CONDA_PROMPT_MODIFIER': '(chatbot) ', 'CONDA_PYTHON_EXE': '/Users/admin/opt/miniconda3/bin/python', 'CONDA_SHLVL': '2', 'HOME': '/Users/admin', 'LOGNAME': 'admin', 'M2_HOME': '/Users/admin/Downloads/apache-maven-3.9.1/', 'MallocNanoZone': '0', 'OLDPWD': '/', 'ORIGINAL_XDG_CURRENT_DESKTOP': 'undefined', 'PATH': '/Users/admin/opt/miniconda3/envs/chatbot/bin:/Users/admin/opt/miniconda3/condabin:/Users/admin/PycharmProjects/gatewaydemo/backend/env/bin:/Users/admin/Downloads/apache-maven-3.9.1/apache-maven/src/bin:/Library/PostgreSQL/15/bin:/Users/admin/Documents/flutter/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Users/admin/Documents/flutter/bin:/Library/Apple/usr/bin:/Users/admin/Library/Android/sdk/platform-tools:/Applications/Google Chrome.app/Contents/MacOS/Google Chrome:/Users/admin/Downloads/kafk

In [8]:
vectorstore = EmbeddingsPDF().get_embeddings()

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=None  # Remove quantization for now, add it back later if needed
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

In [6]:
llm = HuggingFacePipeline(
    pipeline=pipeline,
    )

In [9]:
template = """
[INST] <>
Act as an Insurance expert. Use the following information to answer the question at the end.
<>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [10]:
query = "What is Vitality?"
result_ = qa_chain(
    query
)
result = result_["result"].strip()


display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [11]:
user_question = "What is Vitality?"

In [12]:
response = conversation_chain({"question": user_question})

/Users/admin/opt/miniconda3/envs/chatbot/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}